# Ingest Gene Expression Data from TCGA, TARGET and GTEX

Download gene expression data from the [UCSC Xena Toil re-compute dataset](https://xenabrowser.net/datapages/?host=https://toil.xenahubs.net), wrangle, enrich and store in an hdf5 file for machine learning. This dataset comprises gene expression data for nineteen thousand tumor and normal samples processed using the exact same genomics pipeline and therefore can be compared to each other.

Each sample consists of a float32 vector, log2(TPM+0.001) normalized, of gene expression for each of ~60k genes. Associated with these data is clinical information on each sample such as type (tumor vs. normal), primary site (where the sample came from in the human body) etc... We use this information to label the samples normal/0 vs. tumor/1 as well as to provide additional information for visualization and interpretation of models.

The resulting hdf5 file, tumornormal.h5, contains the following datasets:

* **X_train, X_test:** float32 gene expression train/test sets split via stratification on class (primary site)
* **y_train, y_test:** Binary 0=Normal, 1=Tumor label for binary classification machine learning
* **features:** Hugo gene names for each feature in X
* **labels:** Labels for y binary values ie "Normal" = 0 and "Tumor" = 1
* **classes_train, classes_test:** Class id (integer) for each sample - useful for visualization when clustering
* **class_labels:** Text corresponding to each class id (ie disease) - useful as legend when visualizing

In [2]:
import os
import requests
import numpy as np
import pandas as pd
import h5py

In [31]:
%%time
"""
Download expression data from Xena and save in an hdf5 file. This can take around 
30 minutes between the download and the conversion from tsv into float32 dataframes
We download manually vs. passing read_csv a url directly as the latter times
out with this size file. Note we invert the expression matrix to conform 
to machine learning where rows are samples vs. gene expression files where 
rows are features (gene, transcript, or variant) and columns are 
instances (per sample or cell expression levels)
"""
if not os.path.exists("data"):
    os.makedirs("data")
    
if not os.path.exists("data/TcgaTargetGtex_rsem_gene_tpm.gz"):
    print("Downloading TCGA, TARGET and GTEX expression data from UCSC Xena")
    r = requests.get("https://toil.xenahubs.net/download/TcgaTargetGtex_rsem_gene_tpm.gz")
    r.raise_for_status()
    with open("data/TcgaTargetGtex_rsem_gene_tpm.gz", "wb") as f:
        for chunk in r.iter_content(32768):
            f.write(chunk)

if not os.path.exists("data/TcgaTargetGtex_rsem_gene_tpm.hd5"):
    print("Converting expression to dataframe and storing in hdf5 file")
    expression = pd.read_csv("data/TcgaTargetGtex_rsem_gene_tpm.gz", 
                             sep="\t", index_col=0).dropna().astype(np.float32)
    expression.to_hdf("data/TcgaTargetGtex_rsem_gene_tpm.hd5", "expression", mode="w", format="fixed")

expression = pd.read_hdf("data/TcgaTargetGtex_rsem_gene_tpm.hd5", "expression").sort_index(axis=1)
print("expression: samples={} genes={}".format(*expression.shape))

expression: samples=60498 genes=19260
CPU times: user 2.75 s, sys: 5.23 s, total: 7.98 s
Wall time: 7.97 s


In [33]:
expression.head()

,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,GTEX-1117F-2226-SM-5N9CH,GTEX-1117F-2426-SM-5EGGH,GTEX-1117F-2826-SM-5GZXL,GTEX-1117F-3026-SM-5GZYU,...,TCGA-ZR-A9CJ-01,TCGA-ZS-A9CD-01,TCGA-ZS-A9CE-01,TCGA-ZS-A9CF-01,TCGA-ZS-A9CF-02,TCGA-ZS-A9CG-01,TCGA-ZT-A8OM-01,TCGA-ZU-A8S4-01,TCGA-ZU-A8S4-11,TCGA-ZX-AA5X-01
sample,,,,,,,,,,,,,,,,,,,,,
ENSG00000242268.2,-9.9658,-9.9658,-9.9658,-1.2481,-3.8160,-1.7809,-9.9658,-9.9658,-3.6259,-9.9658,...,-9.9658,-4.6082,-9.9658,-9.9658,-4.6082,-9.9658,-3.6259,-9.9658,-9.9658,-9.9658
ENSG00000259041.1,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658
ENSG00000270112.3,-4.2934,0.0014,-9.9658,-5.5735,0.3573,-9.9658,-6.5064,-5.0116,-9.9658,-5.0116,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-6.5064
ENSG00000167578.16,5.1190,4.1277,4.4067,5.6860,4.0357,4.6849,4.5009,5.3954,4.9402,5.4683,...,4.1780,4.5547,3.6737,4.9331,3.6254,3.7646,5.5201,5.4216,3.3647,4.7991
ENSG00000278814.1,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658


In [34]:
"""
Feature labels are ensemble ids, convert to hugo gene names for use in interpreting
hidden layers in any trained models as they are better known to most bioinformaticians 
and clinicians. We're using an assembled table from John Vivian @ UCSC here. Another
option would be ftp://ftp.ebi.ac.uk/pub/databases/genenames/new/tsv/hgnc_complete_set.txt
"""
ensemble_to_hugo = pd.read_table(
    "https://github.com/jvivian/docker_tools/blob/master/gencode_hugo_mapping/attrs.tsv?raw=true",
    index_col=0)
ensemble_to_hugo.head()

,geneName,geneType,geneStatus,transcriptId,transcriptName,transcriptType,transcriptStatus,havanaGeneId,havanaTranscriptId,ccdsId,level,transcriptClass
geneId,,,,,,,,,,,,
ENSG00000223972.5,DDX11L1,transcribed_unprocessed_pseudogene,KNOWN,ENST00000456328.2,DDX11L1-002,processed_transcript,KNOWN,OTTHUMG00000000961.2,OTTHUMT00000362751.1,NaN,2,pseudo
ENSG00000223972.5,DDX11L1,transcribed_unprocessed_pseudogene,KNOWN,ENST00000450305.2,DDX11L1-001,transcribed_unprocessed_pseudogene,KNOWN,OTTHUMG00000000961.2,OTTHUMT00000002844.2,NaN,2,pseudo
ENSG00000227232.5,WASH7P,unprocessed_pseudogene,KNOWN,ENST00000488147.1,WASH7P-001,unprocessed_pseudogene,KNOWN,OTTHUMG00000000958.1,OTTHUMT00000002839.1,NaN,2,pseudo
ENSG00000278267.1,MIR6859-1,miRNA,KNOWN,ENST00000619216.1,MIR6859-1-201,miRNA,KNOWN,NaN,NaN,NaN,3,nonCoding
ENSG00000243485.3,RP11-34P13.3,lincRNA,KNOWN,ENST00000473358.1,RP11-34P13.3-001,lincRNA,KNOWN,OTTHUMG00000000959.2,OTTHUMT00000002840.1,NaN,2,nonCoding


In [35]:
hugo = ensemble_to_hugo[~ensemble_to_hugo.index.duplicated(keep='first')].reindex(expression.index.values)["geneName"].fillna("")
# Make sure we end up with the order of features being identical as some ensemble id's
# map to the same hugo gene id
assert(expression.index.equals(hugo.index))
# Change index from Ensembl to Hugo
expression_hugo = expression
expression_hugo.index = hugo.values

In [36]:
expression_hugo_mean = expression_hugo.groupby(expression.index).mean()

In [37]:
# Replace duplicate Hugo values with their mean and invert so matrix is in machine learning rows = samples
X = expression_hugo_mean.T
X.head()

,,5S_rRNA,5_8S_rRNA,7SK,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,...,snoU2-30,snoU2_19,snoU83B,snoZ196,snoZ278,snoZ40,snoZ6,snosnR66,uc_338,yR211F11.2
GTEX-1117F-0226-SM-5GZZ7,-9.9658,-9.9658,-9.9658,-9.965800,4.4595,0.9343,-5.0116,7.5126,0.8164,-2.1140,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-7.911678,-9.9658
GTEX-1117F-0426-SM-5EGHI,-9.9658,-9.9658,-9.9658,-9.965800,1.1512,-1.2828,-6.5064,6.0777,-2.3147,0.5568,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-8.490334,-9.9658
GTEX-1117F-0526-SM-5EGHJ,-9.9658,-9.9658,-9.9658,-9.965800,5.2411,0.8488,-6.5064,10.0319,0.1257,-1.1172,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-8.653133,-9.9658
GTEX-1117F-0626-SM-5N9CS,-9.9658,-9.9658,-9.9658,-9.965800,5.4758,2.6325,-9.9658,9.7572,1.7702,-1.8836,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.231703,-9.9658
GTEX-1117F-0726-SM-5GIEN,-9.9658,-9.9658,-9.9658,-5.231167,4.5534,1.3051,-9.9658,7.7931,-0.0725,-2.2447,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.019897,-9.9658


In [38]:
# Read in the sample labels from Xena ie clinical/phenotype information on each sample
if not os.path.exists("data/TcgaTargetGTEX_phenotype.txt.gz"):
    with open("data/TcgaTargetGTEX_phenotype.txt.gz", "wb") as f:
        f.write(requests.get("https://toil.xenahubs.net/download/TcgaTargetGTEX_phenotype.txt.gz").content)

Y = pd.read_table("data/TcgaTargetGTEX_phenotype.txt.gz", compression="gzip",
                  header=0, names=["id", "category", "disease", "primary_site", "sample_type", "gender", "study"],
                  sep="\t", encoding="ISO-8859-1", index_col=0, dtype="str").sort_index(axis=0)

# Compute and add a tumor/normal column - TCGA and TARGET have some normal samples, GTEX is all normal.
Y["tumor_normal"] = Y.apply(
    lambda row: "Normal" if row["sample_type"] in ["Cell Line", "Normal Tissue", "Solid Tissue Normal"]
    else "Tumor", axis=1)

Y[0:100:4000].head()

,category,disease,primary_site,sample_type,gender,study,tumor_normal
id,,,,,,,
GTEX-1117F-0226-SM-5GZZ7,Adipose - Subcutaneous,Adipose - Subcutaneous,Adipose Tissue,Normal Tissue,Female,GTEX,Normal


In [39]:
Y.describe()

,category,disease,primary_site,sample_type,gender,study,tumor_normal
count,19130,19130,19126,19131,18972,19131,19131
unique,93,93,46,17,2,3,2
top,Breast Invasive Carcinoma,Breast Invasive Carcinoma,Brain,Primary Tumor,Male,TCGA,Tumor
freq,1212,1212,1846,9185,10456,10535,10531


In [40]:
# Use the tissue location as the class label for the purposes of stratification
class_attribute = "primary_site"

# Tumor vs. Normal is the binary attribute we'll use to train on
label_attribute = "tumor_normal"

In [41]:
# Remove rows where the class is null or the sample is missing
Y_not_null = Y[pd.notnull(Y[class_attribute])]
intersection = X.index.intersection(Y_not_null.index)
X_clean = X[X.index.isin(intersection)]
Y_clean = Y[Y.index.isin(intersection)]

# Make sure the label and example samples are in the same order
assert(X_clean.index.equals(Y_clean.index))

print(intersection.shape[0], "samples with non-null labels")

19126 samples with non-null labels


In [42]:
# Convert tumor/normal labels to binary 1/0
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_binary = encoder.fit_transform(Y_clean["tumor_normal"])

In [43]:
# Convert classes into numbers
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(Y_clean[class_attribute].values)
classes = encoder.transform(Y_clean[class_attribute])
print("Total classes for stratification:", len(set(classes)))
class_labels = encoder.classes_

Total classes for stratification: 46


In [44]:
%%time
# Split into stratified training and test sets based on classes (i.e. tissue type) so that we have equal
# proportions of each tissue type in the train and test sets
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(X_clean.values, Y_clean[class_attribute]):
    X_train, X_test = X.values[train_index], X_clean.values[test_index]
    y_train, y_test = y_binary[train_index], y_binary[test_index]
    classes_train, classes_test = classes[train_index], classes[test_index]
    sample_labels_train, sample_labels_test = X.index[train_index], X.index[test_index]

CPU times: user 1 s, sys: 1.76 s, total: 2.76 s
Wall time: 3.02 s


In [45]:
"""
Write to an h5 file for training (see above for details on each dataset)
"""
with h5py.File("data/tumor_normal.h5", "w") as f:
    f.create_dataset('X_train', X_train.shape, dtype='f')[:] = X_train
    f.create_dataset('X_test', X_test.shape, dtype='f')[:] = X_test
    f.create_dataset('y_train', y_train.shape, dtype='i')[:] = y_train
    f.create_dataset('y_test', y_test.shape, dtype='i')[:] = y_test
    f.create_dataset('classes_train', y_train.shape, dtype='i')[:] = classes_train
    f.create_dataset('classes_test', y_test.shape, dtype='i')[:] = classes_test
    f.create_dataset('features', X_clean.columns.shape, 'S10', 
                     [l.encode("ascii", "ignore") for l in X_clean.columns.values])
    f.create_dataset('labels', (2, 1), 'S10', 
                     [l.encode("ascii", "ignore") for l in ["Normal", "Tumor"]])
    f.create_dataset('class_labels', (len(class_labels), 1), 'S10', 
                     [l.encode("ascii", "ignore") for l in class_labels])

In [46]:
import matplotlib.pyplot as pyplot
pyplot.hist(classes_train, alpha=0.5, label='Train')
pyplot.hist(classes_test, alpha=0.5, label='Test')
pyplot.legend(loc='upper right')
pyplot.title("Class (Primary Site) distribution between train and test")
pyplot.show()